In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from burbankai.core import *


In [ ]:
#| hide
from burbankai.core import *
from burbankai.trait import *

import torch
import attr

# burbankai

> Pytorch Breeding

A pytorch software for simulating breeding programs. With GPU.

## Install

```sh
pip install burbankai
```

## How to use

First, define the genome of your crop

In [ ]:
ploidy = 2
number_chromosomes = 10
loci_per_chromosome = 100
n_founders = 50
genetic_map = create_uniform_genetic_map(number_chromosomes,loci_per_chromosome)
crop_genome = Genome(ploidy, number_chromosomes, loci_per_chromosome, genetic_map)
founder_pop = create_random_founder_pop(crop_genome , n_founders)


qtl_map = select_qtl_loci(20, crop_genome)
marker_fx = generate_marker_effects(qtl_map)

founder_genetic_variance = calculate_genetic_variance(founder_pop,marker_fx,crop_genome)
scaled_marker_fx = scale_marker_effects(marker_fx, founder_genetic_variance, 0.5)